# 주의사항
1. Colab 에서 작성
1. 데이터셋 경로는 /content/drive/MyDrive/KISTI/new_mush/ 를 기준으로 작성
1. data_gen 함수는 실행시마다 augmetation 데이터를 생성하므로 주의

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#CPU - GPU 정보
#!cat /proc/cpuinfo
#!nvidia-smi

In [2]:
#preprocess.
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import style
from PIL import Image
import random as rn
from matplotlib import font_manager, rc


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

%matplotlib inline  

In [13]:
# Data augmentation
# 실행시 마다 이미지가 생성 되므로 필요한만큼만 실행
# 실행 후에는 Colab에 생성된 이미지가 반영될 때까지 시간 필요 + 코랩 폴더 새로고침할 것
def data_gen(mush_type):
  data_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30, 
                                   # shear_range=5.5,  
                                   # width_shift_range=0.1,
                                   # height_shift_range=0.1,
                                   brightness_range = [0.2,1],
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   #fill_mode='nearest'
                                   ) 
 
 
  filename_in_dir = [] 
 
  for root, dirs, files in os.walk('/content/drive/MyDrive/KISTI/new_mush/{}'.format(mush_type)):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
 
  for file_image in filename_in_dir:
    #print file_image
    img = load_img(file_image) 
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
 
    i = 0
 
    for batch in data_datagen.flow(x,save_to_dir='/content/drive/MyDrive/KISTI/new_mush/{}'.format(mush_type), save_prefix='augmented', save_format='jpg'):
        i += 1
        if i > 3: # 이미지당 3장씩 augmentation
            break

for mush_type in mush_types:
  data_gen(mush_type)

In [5]:
#class_names = os.listdir('./mush_images/')
#class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
#print(class_names_label)
print(os.listdir('/content/drive/MyDrive/KISTI/new_mush/'))

['개암버섯', '갈황색미치광이버섯', '뽕나무버섯', '곰보버섯', '달걀버섯', '개나리광대버섯', '노란꼭지버섯', '노란난버섯', '노란다발버섯', '붉은싸리버섯', '노란싸리버섯', '싸리버섯', '노루궁뎅이버섯', '혈색무당버섯', '표고버섯']


In [3]:
X = []
Z = []
IMG_SIZE = 150

In [18]:
def assign_label(img,flower_type):
    return flower_type

def make_train_data(mushroom_type, DIR):
    for img in tqdm(os.listdir(DIR)):
        label = mushroom_type # ? assign_label(img,flower_type) ??
        path = os.path.join(DIR, img) # DIR + '/' + img 
        #print(path)
        ff = np.fromfile(path, np.uint8)
        img = cv2.imdecode(ff, cv2.IMREAD_COLOR)
        #img = cv2.imread(path, cv2.IMREAD_COLOR) # 한글 경로 문제로 사용 불가, 위 두줄로 대체
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

In [6]:
mush_types = os.listdir('/content/drive/MyDrive/KISTI/new_mush/')

for mush_type in mush_types:
    print("\nLoading data : {}".format(mush_type))
    mush_dir = '/content/drive/MyDrive/KISTI/new_mush/' + mush_type
    #print(mush_dir)
    make_train_data(mush_type, mush_dir)

print(len(X))

  0%|          | 0/905 [00:00<?, ?it/s]

Loading data : 개암버섯


  0%|          | 0/885 [00:00<?, ?it/s]

Loading data : 갈황색미치광이버섯


  0%|          | 0/742 [00:00<?, ?it/s]

Loading data : 개나리광대버섯


100%|██████████| 742/742 [05:55<00:00,  2.09it/s]

2532


In [ ]:
#font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
#rc('font', family=font_name)

sub_X = 5
sub_Y = 5
fig,ax=plt.subplots(sub_Y,sub_X)
fig.set_size_inches(15,15)
for i in range(sub_Y):
    for j in range (sub_X):
        l=rn.randint(0,len(Z))
        b, g, r = cv2.split(X[i])   # img파일을 b,g,r로 분리 + 위치 바꿈
        img2 = cv2.merge([r,g,b]) 
        ax[i,j].imshow(img2)
        ax[i,j].set_title('Mushroom: '+ Z[l])
        
plt.tight_layout()

In [7]:
le = LabelEncoder()
Y = le.fit_transform(Z)
Y = to_categorical(Y,len(mush_types)) # One-hot Encoding
print(Y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [8]:
X = np.array(X)
X = X/255
#print(X)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [10]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [ ]:
'''
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(len(mush_types), activation = "softmax"))
'''

In [11]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(mush_types), activation = "softmax"))


In [12]:
batch_size=128
epochs=100

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [22]:
'''
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)
'''

In [13]:
#optimizer = Adam(lr=0.001)
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0

In [15]:
'''
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size,
                              callbacks = [learning_rate_reduction])
'''
History = model.fit(x = x_train, y = y_train,
                    epochs = epochs,
                    validation_data = (x_test,y_test))

Epoch 1/100
60/60 [==============================] - 14s 93ms/step - loss: 3.4502 - accuracy: 0.3623 - val_loss: 1.0707 - val_accuracy: 0.3791
Epoch 2/100
60/60 [==============================] - 5s 77ms/step - loss: 1.1081 - accuracy: 0.3813 - val_loss: 1.0990 - val_accuracy: 0.3065
Epoch 3/100
60/60 [==============================] - 5s 78ms/step - loss: 1.0970 - accuracy: 0.3977 - val_loss: 1.0504 - val_accuracy: 0.4471
Epoch 4/100
60/60 [==============================] - 5s 78ms/step - loss: 1.0457 - accuracy: 0.4608 - val_loss: 0.9796 - val_accuracy: 0.5134
Epoch 5/100
60/60 [==============================] - 5s 79ms/step - loss: 0.9197 - accuracy: 0.5618 - val_loss: 0.8961 - val_accuracy: 0.5608
Epoch 6/100
60/60 [==============================] - 5s 79ms/step - loss: 0.8191 - accuracy: 0.6263 - val_loss: 0.9847 - val_accuracy: 0.6082
Epoch 7/100
60/60 [==============================] - 5s 79ms/step - loss: 0.7566 - accuracy: 0.6876 - val_loss: 1.3475 - val_accuracy: 0.5134
Epoch

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
model.predict()